In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text

# PREPROCESSING THE DATA

#### DATASETS:
* itc_final_posts (excel) - money control posts divided into spam and legit
* ITC.NS (excel) - ITC stock market data collected from yahoo finance

In [2]:
df_itc = pd.read_csv(r"itc/ITC.NS.csv",
                     parse_dates = ["Date"],
                     index_col = ["Date"])
df_itc.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-31,203.766663,206.266663,202.066666,205.100006,162.141357,13790790.0
2013-02-01,205.166672,206.633331,203.033340,206.266663,163.063675,9906601.0
2013-02-04,206.266663,207.266663,204.166672,204.833328,161.930557,11610360.0
2013-02-05,203.333328,204.466660,201.333328,201.633331,159.400803,7980306.0
2013-02-06,202.233337,203.933334,200.733337,203.233337,160.665695,12331005.0


In [3]:
df_itc_posts = pd.read_excel("itc/itc_final_posts.xlsx")
df_itc_posts.head()

,Unnamed: 0,Messages,Time Stamp,Spam
0,0,NO ISSUE IN MANAGEMENT THEY ARE FOCUSED ON BUS...,2022-02-10 19:23:04,1.0
1,1,Adani Wilmar up 72% in just 3 sessions from li...,2022-02-10 16:12:41,1.0
2,2,"Sure, one day this company will be taken over ...",2022-02-10 17:53:08,1.0
3,3,Timeless financial quotes give investors a bet...,2022-02-10 19:43:23,1.0
4,4,"Positive Rally in Nifty, No Rally in ITC and N...",2022-02-10 19:28:39,0.0


#### STEPS TO PERFORM:
* Removing unnecessary columns
* Removing spam posts
* Getting the correct dates
* Sliding a window of 7 days and preparing the data

#### Removing unnecessary columns

In [4]:
df_itc_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
df_itc_posts.head()

,Messages,Time Stamp,Spam
0,NO ISSUE IN MANAGEMENT THEY ARE FOCUSED ON BUS...,2022-02-10 19:23:04,1.0
1,Adani Wilmar up 72% in just 3 sessions from li...,2022-02-10 16:12:41,1.0
2,"Sure, one day this company will be taken over ...",2022-02-10 17:53:08,1.0
3,Timeless financial quotes give investors a bet...,2022-02-10 19:43:23,1.0
4,"Positive Rally in Nifty, No Rally in ITC and N...",2022-02-10 19:28:39,0.0


#### Removing spam posts

In [5]:
df_itc_posts = df_itc_posts[df_itc_posts.Spam==0.0]
df_itc_posts.drop(["Spam"],axis=1,inplace=True)
df_itc_posts.head()

,Messages,Time Stamp
4,"Positive Rally in Nifty, No Rally in ITC and N...",2022-02-10 19:28:39
7,Strong Long Term Fundamental Strength with an ...,2022-02-10 15:59:02
8,after results today some kind up sides movemen...,2022-02-10 15:43:25
9,do not misguide people in this common forum ev...,2022-02-10 15:42:33
14,"Positive Rally in Nifty, No Rally in ITC and N...",2022-02-10 19:28:39


#### Getting the correct dates

* ITC.NS has stock market data from 31-01-2013 to 11-02-2022 and itc_final_posts has data from 12-11-2021 to 10-02-2022.
* 12-11-2021 has only one post. Hence, the price on 14-11-2021 can be predicted using posts from 13-11-2021. But 14-11-2021 is   a sunday. Hence, the price from 15-11-2021 can be predicted

#### Sliding a window of 7 days and preparing the data

In [7]:
from stock_helper import prepare_data
x,y = prepare_data(df_itc)
x

,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACDh_12_26_9,RSI_14,AROONOSC_14,Close + 1,BBL_20_2.0 + 1,...,AROONOSC_14 + 6,Close + 7,BBL_20_2.0 + 7,BBM_20_2.0 + 7,BBU_20_2.0 + 7,BBB_20_2.0 + 7,BBP_20_2.0 + 7,MACDh_12_26_9 + 7,RSI_14 + 7,AROONOSC_14 + 7
Date,,,,,,,,,,,,,,,,,,,,,
2013-04-01,191.946030,200.301666,208.657303,8.343053,0.769180,0.826599,59.372993,92.857140,206.300003,191.646591,...,85.714287,204.199997,190.924728,197.585007,204.245285,6.741683,0.996601,1.614461,64.485970,78.571426
2013-04-02,192.759949,200.831665,208.903381,8.038295,0.743752,0.702402,59.263271,78.571426,204.800003,191.946030,...,85.714287,204.800003,190.519272,197.824997,205.130737,7.386054,0.977365,1.593910,65.764305,85.714287
2013-04-03,193.529572,201.335007,209.140427,7.753670,0.739043,0.601454,59.979939,78.571426,204.766663,192.759949,...,85.714287,203.333328,190.415604,198.104996,205.794388,7.762946,0.839970,1.414442,60.072098,85.714287
2013-04-04,194.260071,201.566666,208.873260,7.249799,0.372261,0.157366,44.797306,71.428574,205.066666,193.529572,...,85.714287,203.733337,190.666824,198.558334,206.449844,7.948807,0.827884,1.260681,61.061985,85.714287
2013-04-05,195.124146,201.708328,208.292526,6.528425,-0.070179,-0.487384,35.015064,-35.714287,199.699997,194.260071,...,92.857140,201.600006,191.053146,198.883331,206.713516,7.874147,0.673474,0.966548,53.451057,85.714287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-07,210.069550,222.669998,235.270447,11.317599,0.798799,1.845161,60.010979,50.000000,234.300003,209.845398,...,-35.714287,214.600006,212.705643,219.389999,226.074356,6.093582,0.141701,-0.658273,40.944489,-35.714287
2022-02-08,209.922806,223.067505,236.212204,11.785402,0.811247,1.664053,61.201462,50.000000,230.199997,210.069550,...,-35.714287,217.600006,212.912766,219.470001,226.027237,5.975522,0.357410,-0.431872,46.709850,-35.714287
2022-02-09,209.990723,223.477493,236.964279,12.069926,0.747372,1.382966,59.212685,50.000000,231.250000,209.922806,...,28.571428,220.199997,213.046463,219.577499,226.108536,5.948732,0.547657,-0.090360,51.160023,-35.714287


In [8]:
y

Date
2013-04-01    204.800003
2013-04-02    204.766663
2013-04-03    205.066666
2013-04-04    199.699997
2013-04-05    194.199997
                 ...    
2022-02-07    230.199997
2022-02-08    231.250000
2022-02-09    230.149994
2022-02-10    232.250000
2022-02-11    232.449997
Name: Close, Length: 2149, dtype: float32

In [9]:
final_x = x[np.datetime64("2021-11-13"):]
final_y = y[np.datetime64("2021-11-13"):]

In [10]:
df_itc_posts = df_itc_posts[::-1]
df_itc_posts

,Messages,Time Stamp
2045,ITC safe investment bet lowest PE in FMCG sect...,2021-11-12 21:31:26
2042,"Aravachan, The expected price Target for ITC t...",2021-11-13 18:02:15
2040,This is only buy for investment purpose for 25...,2021-11-13 18:03:55
2037,Tips for Monday....buy ITC at 232 sl 230 and t...,2021-11-13 18:48:36
2035,"ITC Infotech eyes Rs 3,000 crore revenue mark ...",2021-11-13 19:00:12
...,...,...
14,"Positive Rally in Nifty, No Rally in ITC and N...",2022-02-10 19:28:39
9,do not misguide people in this common forum ev...,2022-02-10 15:42:33
8,after results today some kind up sides movemen...,2022-02-10 15:43:25
7,Strong Long Term Fundamental Strength with an ...,2022-02-10 15:59:02


In [11]:
final_posts = df_itc_posts[7:]
final_posts

,Messages,Time Stamp
2030,ITC safe investment bet lowest PE in FMCG sect...,2021-11-14 07:31:06
2028,ITC safe investment bet lowest PE in FMCG sect...,2021-11-14 09:20:37
2025,ITC Infotech is open for inorganic growth. Don...,2021-11-13 23:20:56
2023,The bonus may be announced by ITC in 2022. So ...,2021-11-14 10:35:23
2019,Technical and financial analysis shows a stron...,2021-11-14 20:39:57
...,...,...
14,"Positive Rally in Nifty, No Rally in ITC and N...",2022-02-10 19:28:39
9,do not misguide people in this common forum ev...,2022-02-10 15:42:33
8,after results today some kind up sides movemen...,2022-02-10 15:43:25
7,Strong Long Term Fundamental Strength with an ...,2022-02-10 15:59:02


In [12]:
len(final_x),len(final_posts)

(63, 614)

# CALCULATING SENTIMENTS SCORE

In [13]:
sent_model = tf.keras.models.load_model("final_bert")
sent_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 preprocessor (KerasLayer)      {'input_word_ids':   0           ['inputs[0][0]']                 
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

#### Removing duplicates

In [14]:
final_posts.drop_duplicates(subset=['Messages'])

,Messages,Time Stamp
2030,ITC safe investment bet lowest PE in FMCG sect...,2021-11-14 07:31:06
2025,ITC Infotech is open for inorganic growth. Don...,2021-11-13 23:20:56
2023,The bonus may be announced by ITC in 2022. So ...,2021-11-14 10:35:23
2019,Technical and financial analysis shows a stron...,2021-11-14 20:39:57
2018,Itc is biggest beneficiary for boom in sales t...,2021-11-14 20:41:18
...,...,...
38,after results today some kind up sides movemen...,2022-02-10 15:43:25
37,Strong Long Term Fundamental Strength with an ...,2022-02-10 15:59:02
36,M Ambani Or G Adani should buy controlling sta...,2022-02-10 17:16:29
34,"Positive Rally in Nifty, No Rally in ITC and N...",2022-02-10 19:28:39


In [15]:
sentiments = []
indices = []
prev = np.datetime64("2015-11-12 21:31:26")
for i in final_y.index:
    total=0
    cnt=0
    for j in final_posts.itertuples():
        k,msg,time = j
        indices.append(k)
        if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
            total += tf.squeeze(sent_model.predict([msg])).numpy()
            print(f'Score: {tf.squeeze(sent_model.predict([msg])).numpy()}')
            cnt+=1
    prev = np.datetime64(i)
    if(cnt==0):
        sentiments.append(0)
    else:
        sentiments.append(total/cnt)

Score: 0.9444242715835571
Score: 0.9444242715835571
Score: 0.9895212650299072
Score: 0.9904130101203918
Score: 0.9859876036643982
Score: 0.9878939986228943
Score: 0.9913119077682495
Score: 0.56297367811203
Score: 0.9855405688285828
Score: 0.9444242715835571
Score: 0.9900887608528137
Score: 0.9864593744277954
Score: 0.8831420540809631
Score: 0.9906072616577148
Score: 0.9881131052970886
Score: 0.9378788471221924
Score: 0.9728671312332153
Score: 0.9444242715835571
Score: 0.9893563389778137
Score: 0.7976952791213989
Score: 0.9805727005004883
Score: 0.8743585348129272
Score: 0.9097331762313843
Score: 0.12076428532600403
Score: 0.9446114897727966
Score: 0.9894975423812866
Score: 0.938491940498352
Score: 0.9903286099433899
Score: 0.9900509715080261
Score: 0.9887735843658447
Score: 0.9857649207115173
Score: 0.8578930497169495
Score: 0.9444242715835571
Score: 0.9726449847221375
Score: 0.9897757768630981
Score: 0.9899868369102478
Score: 0.9444242715835571
Score: 0.9891798496246338
Score: 0.09316

Score: 0.9847087860107422
Score: 0.06511615216732025
Score: 0.9884053468704224
Score: 0.8030979633331299
Score: 0.15169541537761688
Score: 0.9802668690681458
Score: 0.9874778985977173
Score: 0.9471572637557983
Score: 0.9907183051109314
Score: 0.08679325133562088
Score: 0.04434254765510559
Score: 0.9205825924873352
Score: 0.07107432931661606
Score: 0.9907665252685547
Score: 0.7942423224449158
Score: 0.18081220984458923
Score: 0.9892515540122986
Score: 0.9900057911872864
Score: 0.9842122197151184
Score: 0.9724715948104858
Score: 0.9386971592903137
Score: 0.035825181752443314
Score: 0.6403425335884094
Score: 0.08066266775131226
Score: 0.9788405895233154
Score: 0.021515879780054092
Score: 0.9305002093315125
Score: 0.9889113903045654
Score: 0.704558789730072
Score: 0.9879283308982849
Score: 0.9676231741905212
Score: 0.9781097769737244
Score: 0.9715830087661743
Score: 0.9212403297424316
Score: 0.9858099818229675
Score: 0.7468068599700928
Score: 0.7792054414749146
Score: 0.9387190341949463
Sc

# SOLVING THE 0 SENTIMENT SCORE PROBLEM

#### Steps to solve the problem:
* Getting the indices where sentiment score is 0
* Removing those values from the sentiments array and final_x
* Removing the added index column
* Using N-beats model to make predictions on the prepared final_x datset
* Make the final dataset to train final model

#### Getting the indices where sentiment score is 0

In [33]:
zero_index = []
for i,j in enumerate(sentiments):
    if(j==0):
        zero_index.append(i)
zero_index 

[4, 5, 6, 7, 8, 9, 10, 11, 12, 57]

#### Removing those values from the sentiments array and final_x

In [47]:
final_x_zeros = final_x.copy()
final_y_zeros = final_y.copy()
final_y_zeros = final_y_zeros.to_frame()
final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)

In [48]:
final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]

In [49]:
len(final_y_zeros),len(final_x_zeros)

(53, 53)

In [50]:
sentiments = np.delete(sentiments,zero_index)
sentiments

array([0.93790942, 0.94711164, 0.90710024, 0.8002191 , 0.98364121,
       0.34421938, 0.81026217, 0.93727617, 0.85286945, 0.81385339,
       0.91488078, 0.84711482, 0.78174605, 0.78755366, 0.79193413,
       0.88688747, 0.81942778, 0.86818131, 0.57996973, 0.94231537,
       0.81385345, 0.71057868, 0.79632066, 0.71666523, 0.83461632,
       0.75409228, 0.76133595, 0.64567702, 0.74668035, 0.75681433,
       0.90270813, 0.90020501, 0.66226926, 0.91934999, 0.89887066,
       0.89318242, 0.74312352, 0.75146003, 0.76075468, 0.77344375,
       0.86227876, 0.9537311 , 0.79507104, 0.93639518, 0.85952354,
       0.62649698, 0.78937954, 0.98839545, 0.87384896, 0.90128844,
       0.65563229, 0.61601235, 0.81838229])

#### Removing the added index column

In [53]:
final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)

#### Using N-beats model to make predictions on the prepared final_x datset

In [51]:
model_nbeats = tf.keras.models.load_model("nbeats_itc")
model_nbeats.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 71)]         0           []                               
                                                                                                  
 blocks_1 (blocks)              ((None, 71),         861768      ['input_1[0][0]']                
                                 (None, 1))                                                       
                                                                                                  
 subtract (Subtract)            (None, 71)           0           ['input_1[0][0]',                
                                                                  'blocks_1[0][0]']               
                                                                                              

In [59]:
preds = tf.squeeze(model_nbeats.predict(final_x_zeros)).numpy()

#### Make the final dataset to train final model

In [61]:
final_df = pd.DataFrame({"Predictions": preds,
                        "Sentiments":sentiments})

In [62]:
len(final_df),len(final_y_zeros)

(53, 53)

# BUILDING AND TRAINING THE FINAL MODEL

In [63]:
tf.random.set_seed(42)
inputs = tf.keras.Input(shape=(2))
x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(inputs)
x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(x)
x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.models.Model(inputs=inputs,outputs=outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 8)                 24        
                                                                 
 dense_3 (Dense)             (None, 8)                 72        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 177
Trainable params: 177
Non-trainable params: 0
_________________________________________________________________


In [64]:
xtrain,ytrain,xtest,ytest = final_df[:42],final_y_zeros[:42],final_df[42:],final_y_zeros[42:]

In [65]:
model.compile(loss = "mae", optimizer = tf.keras.optimizers.Adam(), metrics=["mae","mse"])
history = model.fit(xtrain,ytrain,
                    epochs = 1000,
                    validation_data=(xtest,ytest), 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                            patience=200, 
                                                            restore_best_weights=True),
                          tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                               patience=100, 
                                                               verbose=1)])

Epoch 1/1000
2/2 [==============================] - 0s 124ms/step - loss: 206.3576 - mae: 206.3576 - mse: 42631.5312 - val_loss: 208.8917 - val_mae: 208.8917 - val_mse: 43671.8438 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 36ms/step - loss: 203.6895 - mae: 203.6895 - mse: 41538.1055 - val_loss: 206.1554 - val_mae: 206.1554 - val_mse: 42535.3555 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 34ms/step - loss: 201.0051 - mae: 201.0051 - mse: 40451.4102 - val_loss: 203.4059 - val_mae: 203.4059 - val_mse: 41408.4492 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 36ms/step - loss: 198.3138 - mae: 198.3138 - mse: 39376.0078 - val_loss: 200.6421 - val_mae: 200.6421 - val_mse: 40290.9766 - lr: 0.0010
Epoch 5/1000
2/2 [==============================] - 0s 34ms/step - loss: 195.6032 - mae: 195.6032 - mse: 38306.2266 - val_loss: 197.8635 - val_mae: 197.8635 - val_mse: 39182.9219 - lr: 0.0010
Epoch 6/1000
2/2 [=====================

Epoch 44/1000
2/2 [==============================] - 0s 32ms/step - loss: 62.6880 - mae: 62.6880 - mse: 3949.2109 - val_loss: 60.2938 - val_mae: 60.2938 - val_mse: 3641.8689 - lr: 0.0010
Epoch 45/1000
2/2 [==============================] - 0s 32ms/step - loss: 57.8907 - mae: 57.8907 - mse: 3368.8169 - val_loss: 55.2758 - val_mae: 55.2758 - val_mse: 3061.4333 - lr: 0.0010
Epoch 46/1000
2/2 [==============================] - 0s 32ms/step - loss: 52.9622 - mae: 52.9622 - mse: 2823.2083 - val_loss: 50.1139 - val_mae: 50.1139 - val_mse: 2516.9324 - lr: 0.0010
Epoch 47/1000
2/2 [==============================] - 0s 36ms/step - loss: 47.8916 - mae: 47.8916 - mse: 2310.4363 - val_loss: 44.8042 - val_mae: 44.8042 - val_mse: 2012.4872 - lr: 0.0010
Epoch 48/1000
2/2 [==============================] - 0s 40ms/step - loss: 42.6771 - mae: 42.6771 - mse: 1840.8806 - val_loss: 39.3425 - val_mae: 39.3425 - val_mse: 1552.4679 - lr: 0.0010
Epoch 49/1000
2/2 [==============================] - 0s 36ms/step

2/2 [==============================] - 0s 28ms/step - loss: 2.4542 - mae: 2.4542 - mse: 11.7311 - val_loss: 1.4395 - val_mae: 1.4395 - val_mse: 2.7378 - lr: 0.0010
Epoch 90/1000
2/2 [==============================] - 0s 41ms/step - loss: 2.4922 - mae: 2.4922 - mse: 11.8467 - val_loss: 1.3112 - val_mae: 1.3112 - val_mse: 2.6529 - lr: 0.0010
Epoch 91/1000
2/2 [==============================] - 0s 39ms/step - loss: 2.5483 - mae: 2.5483 - mse: 12.2336 - val_loss: 1.3298 - val_mae: 1.3298 - val_mse: 2.6391 - lr: 0.0010
Epoch 92/1000
2/2 [==============================] - 0s 40ms/step - loss: 2.5274 - mae: 2.5274 - mse: 12.1357 - val_loss: 1.4265 - val_mae: 1.4265 - val_mse: 2.7100 - lr: 0.0010
Epoch 93/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4606 - mae: 2.4606 - mse: 11.8104 - val_loss: 1.5854 - val_mae: 1.5854 - val_mse: 3.1449 - lr: 0.0010
Epoch 94/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4670 - mae: 2.4670 - mse: 11.7892 - val_loss: 1.

Epoch 135/1000
2/2 [==============================] - 0s 28ms/step - loss: 2.5161 - mae: 2.5161 - mse: 12.0702 - val_loss: 1.3191 - val_mae: 1.3191 - val_mse: 2.6456 - lr: 0.0010
Epoch 136/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.5361 - mae: 2.5361 - mse: 12.2065 - val_loss: 1.3716 - val_mae: 1.3716 - val_mse: 2.6402 - lr: 0.0010
Epoch 137/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4901 - mae: 2.4901 - mse: 11.8915 - val_loss: 1.4830 - val_mae: 1.4830 - val_mse: 2.8619 - lr: 0.0010
Epoch 138/1000
2/2 [==============================] - 0s 28ms/step - loss: 2.4592 - mae: 2.4592 - mse: 11.8359 - val_loss: 1.6679 - val_mae: 1.6679 - val_mse: 3.4304 - lr: 0.0010
Epoch 139/1000
2/2 [==============================] - 0s 28ms/step - loss: 2.5058 - mae: 2.5058 - mse: 11.9032 - val_loss: 1.6016 - val_mae: 1.6016 - val_mse: 3.1963 - lr: 0.0010
Epoch 140/1000
2/2 [==============================] - 0s 36ms/step - loss: 2.4288 - mae: 2.4288 - mse: 11

2/2 [==============================] - 0s 44ms/step - loss: 2.4555 - mae: 2.4555 - mse: 11.7760 - val_loss: 1.5860 - val_mae: 1.5860 - val_mse: 3.1475 - lr: 1.0000e-04
Epoch 180/1000
2/2 [==============================] - 0s 36ms/step - loss: 2.4551 - mae: 2.4551 - mse: 11.7752 - val_loss: 1.5806 - val_mae: 1.5806 - val_mse: 3.1309 - lr: 1.0000e-04
Epoch 181/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4525 - mae: 2.4525 - mse: 11.7638 - val_loss: 1.5629 - val_mae: 1.5629 - val_mse: 3.0786 - lr: 1.0000e-04
Epoch 182/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4463 - mae: 2.4463 - mse: 11.7690 - val_loss: 1.5289 - val_mae: 1.5289 - val_mse: 2.9865 - lr: 1.0000e-04
Epoch 183/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4456 - mae: 2.4456 - mse: 11.7867 - val_loss: 1.5036 - val_mae: 1.5036 - val_mse: 2.9248 - lr: 1.0000e-04
Epoch 184/1000
2/2 [==============================] - 0s 28ms/step - loss: 2.4446 - mae: 2.4446 - ms

Epoch 224/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4488 - mae: 2.4488 - mse: 11.7690 - val_loss: 1.4809 - val_mae: 1.4809 - val_mse: 2.8546 - lr: 1.0000e-04
Epoch 225/1000
2/2 [==============================] - 0s 36ms/step - loss: 2.4469 - mae: 2.4469 - mse: 11.7616 - val_loss: 1.4906 - val_mae: 1.4906 - val_mse: 2.8886 - lr: 1.0000e-04
Epoch 226/1000
2/2 [==============================] - 0s 36ms/step - loss: 2.4448 - mae: 2.4448 - mse: 11.7557 - val_loss: 1.5055 - val_mae: 1.5055 - val_mse: 2.9292 - lr: 1.0000e-04
Epoch 227/1000
2/2 [==============================] - 0s 40ms/step - loss: 2.4452 - mae: 2.4452 - mse: 11.7407 - val_loss: 1.5255 - val_mae: 1.5255 - val_mse: 2.9778 - lr: 1.0000e-04
Epoch 228/1000
2/2 [==============================] - 0s 44ms/step - loss: 2.4413 - mae: 2.4413 - mse: 11.7443 - val_loss: 1.5441 - val_mae: 1.5441 - val_mse: 3.0263 - lr: 1.0000e-04
Epoch 229/1000
2/2 [==============================] - 0s 44ms/step - loss: 2.4460 - m

In [66]:
model.evaluate(xtest,ytest)

1/1 [==============================] - 0s 66ms/step - loss: 1.2398 - mae: 1.2398 - mse: 2.7875


[1.2398127317428589, 1.2398127317428589, 2.7874674797058105]

# Training a conv1d model on the same data

In [72]:
tf.random.set_seed(42)
inputs = tf.keras.Input(shape=(71,1), name="inputs")
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(inputs)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(x)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(x)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.models.Model(inputs, outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 71, 1)]           0         
                                                                 
 conv1d_8 (Conv1D)           (None, 71, 64)            256       
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 35, 64)           0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 35, 64)            12352     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 17, 64)           0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 17, 64)            1235

In [73]:
model.compile(loss = "mae", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["mae","mse"])
history = model.fit(final_x_zeros[:42],final_y_zeros[:42],
                    epochs = 1000,
                    validation_data=(final_x_zeros[42:], final_y_zeros[42:]), 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                            patience=200, 
                                                            restore_best_weights=True),
                          tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                               patience=100, 
                                                               verbose=1)])

Epoch 1/1000
2/2 [==============================] - 1s 617ms/step - loss: 216.8970 - mae: 216.8970 - mse: 47096.8438 - val_loss: 214.8625 - val_mae: 214.8625 - val_mse: 46198.5664 - lr: 1.0000e-04
Epoch 2/1000
2/2 [==============================] - 0s 48ms/step - loss: 208.8444 - mae: 208.8444 - mse: 43668.6562 - val_loss: 207.1177 - val_mae: 207.1177 - val_mse: 42931.1602 - lr: 1.0000e-04
Epoch 3/1000
2/2 [==============================] - 0s 36ms/step - loss: 200.8758 - mae: 200.8758 - mse: 40403.9414 - val_loss: 199.3156 - val_mae: 199.3156 - val_mse: 39760.0391 - lr: 1.0000e-04
Epoch 4/1000
2/2 [==============================] - 0s 32ms/step - loss: 192.9569 - mae: 192.9569 - mse: 37283.7266 - val_loss: 191.7895 - val_mae: 191.7895 - val_mse: 36817.5898 - lr: 1.0000e-04
Epoch 5/1000
2/2 [==============================] - 0s 32ms/step - loss: 185.2302 - mae: 185.2302 - mse: 34358.7188 - val_loss: 184.5834 - val_mae: 184.5834 - val_mse: 34107.8711 - lr: 1.0000e-04
Epoch 6/1000
2/2 [=

2/2 [==============================] - 0s 36ms/step - loss: 4.7773 - mae: 4.7773 - mse: 33.1163 - val_loss: 11.1884 - val_mae: 11.1884 - val_mse: 150.5921 - lr: 1.0000e-04
Epoch 45/1000
2/2 [==============================] - 0s 32ms/step - loss: 4.7716 - mae: 4.7716 - mse: 33.7467 - val_loss: 10.5028 - val_mae: 10.5028 - val_mse: 134.9721 - lr: 1.0000e-04
Epoch 46/1000
2/2 [==============================] - 0s 32ms/step - loss: 4.5159 - mae: 4.5159 - mse: 29.7692 - val_loss: 9.1876 - val_mae: 9.1876 - val_mse: 105.3963 - lr: 1.0000e-04
Epoch 47/1000
2/2 [==============================] - 0s 32ms/step - loss: 4.2299 - mae: 4.2299 - mse: 25.6787 - val_loss: 8.0805 - val_mae: 8.0805 - val_mse: 82.6240 - lr: 1.0000e-04
Epoch 48/1000
2/2 [==============================] - 0s 36ms/step - loss: 4.2612 - mae: 4.2612 - mse: 27.0060 - val_loss: 7.6508 - val_mae: 7.6508 - val_mse: 74.6650 - lr: 1.0000e-04
Epoch 49/1000
2/2 [==============================] - 0s 32ms/step - loss: 4.1218 - mae: 4.12

Epoch 89/1000
2/2 [==============================] - 0s 36ms/step - loss: 2.9191 - mae: 2.9191 - mse: 13.8047 - val_loss: 6.9535 - val_mae: 6.9535 - val_mse: 60.4890 - lr: 1.0000e-04
Epoch 90/1000
2/2 [==============================] - 0s 32ms/step - loss: 3.1504 - mae: 3.1504 - mse: 15.8846 - val_loss: 6.0606 - val_mae: 6.0606 - val_mse: 47.6676 - lr: 1.0000e-04
Epoch 91/1000
2/2 [==============================] - 0s 44ms/step - loss: 3.2438 - mae: 3.2438 - mse: 18.1153 - val_loss: 6.9922 - val_mae: 6.9922 - val_mse: 60.7736 - lr: 1.0000e-04
Epoch 92/1000
2/2 [==============================] - 0s 44ms/step - loss: 3.0289 - mae: 3.0289 - mse: 14.7432 - val_loss: 8.4686 - val_mae: 8.4686 - val_mse: 83.5169 - lr: 1.0000e-04
Epoch 93/1000
2/2 [==============================] - 0s 48ms/step - loss: 2.9726 - mae: 2.9726 - mse: 13.5917 - val_loss: 8.4896 - val_mae: 8.4896 - val_mse: 83.7432 - lr: 1.0000e-04
Epoch 94/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.9209 - ma

Epoch 133/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.5416 - mae: 2.5416 - mse: 11.7647 - val_loss: 7.1446 - val_mae: 7.1446 - val_mse: 59.5045 - lr: 1.0000e-05
Epoch 134/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.5488 - mae: 2.5488 - mse: 11.9330 - val_loss: 7.0887 - val_mae: 7.0887 - val_mse: 58.6917 - lr: 1.0000e-05
Epoch 135/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.5577 - mae: 2.5577 - mse: 12.0445 - val_loss: 7.0860 - val_mae: 7.0860 - val_mse: 58.6357 - lr: 1.0000e-05
Epoch 136/1000
2/2 [==============================] - 0s 34ms/step - loss: 2.5565 - mae: 2.5565 - mse: 12.0470 - val_loss: 7.1288 - val_mae: 7.1288 - val_mse: 59.2302 - lr: 1.0000e-05
Epoch 137/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.5394 - mae: 2.5394 - mse: 11.9326 - val_loss: 7.2195 - val_mae: 7.2195 - val_mse: 60.5234 - lr: 1.0000e-05
Epoch 138/1000
2/2 [==============================] - 0s 31ms/step - loss: 2.532

2/2 [==============================] - 0s 32ms/step - loss: 2.4933 - mae: 2.4933 - mse: 11.2676 - val_loss: 7.4316 - val_mae: 7.4316 - val_mse: 63.3660 - lr: 1.0000e-05
Epoch 178/1000
2/2 [==============================] - 0s 36ms/step - loss: 2.4939 - mae: 2.4939 - mse: 11.2518 - val_loss: 7.4080 - val_mae: 7.4080 - val_mse: 63.0084 - lr: 1.0000e-05
Epoch 179/1000
2/2 [==============================] - 0s 34ms/step - loss: 2.4901 - mae: 2.4901 - mse: 11.3008 - val_loss: 7.3532 - val_mae: 7.3532 - val_mse: 62.1884 - lr: 1.0000e-05
Epoch 180/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4815 - mae: 2.4815 - mse: 11.3781 - val_loss: 7.2606 - val_mae: 7.2606 - val_mse: 60.8243 - lr: 1.0000e-05
Epoch 181/1000
2/2 [==============================] - 0s 35ms/step - loss: 2.4801 - mae: 2.4801 - mse: 11.5047 - val_loss: 7.1692 - val_mae: 7.1692 - val_mse: 59.4950 - lr: 1.0000e-05
Epoch 182/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4847 - mae: 2.4847

Epoch 222/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4472 - mae: 2.4472 - mse: 11.2001 - val_loss: 7.4853 - val_mae: 7.4853 - val_mse: 63.9193 - lr: 1.0000e-05
Epoch 223/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4429 - mae: 2.4429 - mse: 10.9168 - val_loss: 7.5739 - val_mae: 7.5739 - val_mse: 65.2469 - lr: 1.0000e-05
Epoch 224/1000
2/2 [==============================] - 0s 32ms/step - loss: 2.4483 - mae: 2.4483 - mse: 10.8282 - val_loss: 7.5485 - val_mae: 7.5485 - val_mse: 64.8571 - lr: 1.0000e-05
Epoch 225/1000
1/2 [==============>...............] - ETA: 0s - loss: 2.5845 - mae: 2.5845 - mse: 12.0200
Epoch 00225: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
2/2 [==============================] - 0s 32ms/step - loss: 2.4453 - mae: 2.4453 - mse: 10.8787 - val_loss: 7.4617 - val_mae: 7.4617 - val_mse: 63.5513 - lr: 1.0000e-05


In [101]:
model.evaluate(final_x_zeros[42:],final_y_zeros[42:])
model.save("conv1d_itc")

1/1 [==============================] - 0s 27ms/step - loss: 4.3033 - mae: 4.3033 - mse: 40.9895
INFO:tensorflow:Assets written to: conv1d_itc\assets


# BINGO!!! NOW LETS MAKE THE FINAL FUNCTION

In [100]:
def model_scores_comparator(add_time,add_posts,bert,stock,l):
    '''
    Args:
    add_time: address of the file that has time series data of the stock
    add_posts: address of the  file that has all the posts of the stock, labelled as spam or not spam
    bert: address of the bert sentiment analysis model
    stock: name of the stock
    l: list of all models that the target model's results have to be compared to

    Returns:
    A pandas dataframe containing the MAEs and MSEs of all the models
    '''
    import pandas as pd
    import tensorflow as tf
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import tensorflow_hub as hub
    import tensorflow_text as text
    df_stock = pd.read_csv(add_time,
                     parse_dates = ["Date"],
                     index_col = ["Date"])
    df_posts = pd.read_excel(add_posts)
    # removing the unnecessary columns
    df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
    # removing spam posts
    df_posts = df_posts[df_itc_posts.Spam==0.0]
    df_posts.drop(["Spam"],axis=1,inplace=True)
    # sliding a window of 7 days and adding all the TIs
    from stock_helper import prepare_data
    x,y = prepare_data(df_stock)
    # slicing the data
    final_x = x[np.datetime64("2021-11-13"):]
    final_y = y[np.datetime64("2021-11-13"):]
    # reversing the posts data
    df_posts = df_posts[::-1]
    final_posts = df_posts[7:]
    # loading the sentiment analysis model
    sent_model = tf.keras.models.load_model(bert)
    # removing duplicates from the data
    final_posts.drop_duplicates(subset=['Messages'])
    # calculating the sentiments score
    sentiments = []
    prev = np.datetime64("2015-11-12 21:31:26")
    for i in final_y.index:
        total=0
        cnt=0
        for j in final_posts.itertuples():
            _,msg,time = j
            if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
                total += tf.squeeze(sent_model.predict([msg])).numpy()
                cnt+=1
        prev = np.datetime64(i)
        if(cnt==0):
            sentiments.append(0)
        else:
            sentiments.append(total/cnt)
    # getting indices where sentiments score is 0
    zero_index = []
    for i,j in enumerate(sentiments):
        if(j==0):
            zero_index.append(i)
    # removing all the zero values indices
    sentiments = np.delete(sentiments,zero_index)
    final_x_zeros = final_x.copy()
    final_y_zeros = final_y.copy()
    final_y_zeros = final_y_zeros.to_frame()
    final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
    final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)
    final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
    final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]
    # removing the added helper column
    final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
    final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)
    # loading the nbeats model for the given stock
    model_nbeats = tf.keras.models.load_model("nbeats_" + stock)
    # making predictions and building the final dataframe 
    preds = tf.squeeze(model_nbeats.predict(final_x_zeros)).numpy()
    final_df = pd.DataFrame({"Predictions": preds,
                        "Sentiments":sentiments})
    # building and training the target model
    tf.random.set_seed(42)
    inputs = tf.keras.Input(shape=(2))
    x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(inputs)
    x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(x)
    x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(x)
    outputs = tf.keras.layers.Dense(1)(x)
    model = tf.keras.models.Model(inputs=inputs,outputs=outputs)
    
    xtrain,ytrain,xtest,ytest = final_df[:42],final_y_zeros[:42],final_df[42:],final_y_zeros[42:]
    
    model.compile(loss = "mae", optimizer = tf.keras.optimizers.Adam(), metrics=["mae","mse"])
    history = model.fit(xtrain,ytrain,
                        epochs = 1000,
                        validation_data=(xtest,ytest), 
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                                patience=200, 
                                                                restore_best_weights=True),
                              tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                                   patience=100, 
                                                                   verbose=1)])
    target_result = model.evaluate(xtest,ytest)
    dict_res = {
        "Target_model":target_result[1]
    }
    df = pd.DataFrame(dict_res)
    frames = []
    frames.append(df)
    # getting the results of all the other models
    for i in l:
        name = i + "_" + stock
        model = tf.keras.models.load_model(name)
        result = model.evaluate(final_x_zeros[42:], final_y_zeros[42:])
        dict_res = {
         i:result[1]
        }
        df = pd.DataFrame(dict_res)
        frames.append(df)
    return pd.concat(frames, axis=1)

In [103]:
df = model_scores_comparator("itc/ITC.NS.csv", "itc/itc_final_posts.xlsx", 
                             "final_bert", "itc", ["conv1d"])

AttributeError: 'DataFrame' object has no attribute 'Spam'

In [112]:
z = res[0]
dict1 = {
    "f":z
}
g = pd.DataFrame.from_dict(dict1)
g

ValueError: If using all scalar values, you must pass an index